In [ ]:
!pip install kaggle
!pip install pandas matplotlib seaborn

In [ ]:
!pip install gdown

In [ ]:
import os
from google.colab import files

uploaded = files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d shanegerami/ai-vs-human-text

100% 349M/350M [00:08<00:00, 45.6MB/s]
100% 350M/350M [00:08<00:00, 41.1MB/s]


In [ ]:
zip_file = f"ai-vs-human-text.zip"
!unzip -q $zip_file -d data
print("Dataset downloaded and unzipped successfully!")

Dataset downloaded and unzipped successfully!


# Imports

In [ ]:
# Imports
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

In [ ]:
# Ensure necessary NLTK downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Preprocessing

In [ ]:
# Define a comprehensive text preprocessing function
def preprocess_text(text):
    # Lowercase conversion
    text = text.lower()
    # Removal of non-word characters
    text = re.sub(r'\W', ' ', text)
    # Removing extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    text = ' '.join([word for word in words if word not in stop_words])
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in words])
    return text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !cp /content/cleaned.csv /content/drive/MyDrive/DMA

In [ ]:

df = pd.read_csv("/content/drive/MyDrive/DMA/cleaned.csv")

In [ ]:
df.head()

,Unnamed: 0,text,generated
0,0,car car around since became famous 1900s henry...,0.0
1,1,transportation large necessity country worldwi...,0.0
2,2,america love affair vehicle seems cooling say ...,0.0
3,3,often ride car drive one motor vehicle work st...,0.0
4,4,car wonderful thing perhaps one world greatest...,0.0


In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['generated'], test_size=0.2, random_state=42)

In [ ]:
X_train.head()

110501    think facs useful technology could benefit cla...
262291    student create summer project instead teacher ...
159503    average 8thgrade student developed talent play...
193029    holy avocado new computer software come design...
31127     title cowboy rode wave upon time lived cowboy ...
Name: text, dtype: object

In [ ]:
df_zeros = df[df['generated'] == 0]
df_ones = df[df['generated'] == 1]

# Take a random sample of 5000 from each part
sample_zeros = df_zeros.sample(n=10000, random_state=42)
sample_ones = df_ones.sample(n=10000, random_state=42)

# Combine the samples
sample_df = pd.concat([sample_zeros, sample_ones])

# Shuffle the combined sample to mix the 0s and 1s
sample_df = sample_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the distribution of the 'generated' column in the sample
print(sample_df['generated'].value_counts())

# Print sample size
print("Sample size:", len(sample_df))

generated
1.0    10000
0.0    10000
Name: count, dtype: int64
Sample size: 20000


In [ ]:
sample_zeros.head()

,Unnamed: 0,text,generated
260481,260481,curfew keep teenager getting trouble city coun...,0.0
133303,133303,article challenge exploring venus author sugge...,0.0
291551,291551,thp rapid growth thp internet recent decade th...,0.0
451335,451335,electoral college way u united state citizen v...,0.0
461660,461660,technology calculate emotion people pretty muc...,0.0


In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sample_df['text'], sample_df['generated'], test_size=0.2, random_state=42)

In [ ]:
print(X_test)

10650    think se change election popular vote presiden...
2041     article challenge exploring venus effectively ...
8668     throughout year fact year 2002 2014 numerous n...
1114     soday many people go many country easily shey ...
13902    dear principal writing express view possible p...
                               ...                        
4073     far driverless car also think concern driverle...
7442     dear senator writing today express support abo...
9999     often said positive attitude key success lira ...
1870     io single answer question whether successful p...
15196    evening star otherwise known genus dangerous p...
Name: text, Length: 4000, dtype: object


In [ ]:
X_train_cleaned = np.where(pd.isnull(X_train), '', X_train)
X_test_cleaned = np.where(pd.isnull(X_test), '', X_test)

In [ ]:
print(X_test_cleaned)

['think se change election popular vote president united state se keep electoral college even earn individual vote might elected president earned individual vote elected president majority vote se keep electoral college elected president receive majority individual vote president elected electoral college still popular president people voted source 2 say consider state legislature technically responsible picking elector elector could always defy sill people sith electoral college elector chosen might elect shot hey said would sith election popular vote person elected sill always majority country santa president source 1 state state entitled allotment elector equal number member congressional delegation one member house representative plus tso senator state vote others someone running get state vote mean still elected get vote overall elector source 3 say advocate position correct arguing electoral college method democratic modern sense elector elect president people sith electoral coll

In [ ]:
# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
# vectorizer = TfidfVectorizer(ngram_range=(3,3))
X_train_tfidf = vectorizer.fit_transform(X_train_cleaned)
X_test_tfidf = vectorizer.transform(X_test_cleaned)

In [ ]:
# save th tfidf vectorizer
joblib.dump(vectorizer, 'vectorizer_tfidf.joblib')

NameError: name 'joblib' is not defined

In [ ]:
!cp /content/vectorizer_tfidf.joblib /content/drive/MyDrive/DMA/

In [ ]:
import joblib

# Save the TF-IDF transformed features
joblib.dump(X_train_tfidf, 'X_train_tfidf.joblib')
joblib.dump(X_test_tfidf, 'X_test_tfidf.joblib')

In [ ]:
!cp /content/X_train_tfidf.joblib /content/drive/MyDrive/DMA/
!cp /content/X_test_tfidf.joblib /content/drive/MyDrive/DMA/

In [ ]:
print(len(X_train_cleaned))

In [ ]:
# Vectorize text using TF-IDF Ngram
vectorizer_ngram = TfidfVectorizer(ngram_range=(3,3))
X_train_ngram = vectorizer_ngram.fit_transform(X_train_cleaned)
X_test_ngram = vectorizer_ngram.transform(X_test_cleaned)

In [ ]:
# save th ngram vectorizer
joblib.dump(vectorizer_ngram, 'vectorizer_ngram.joblib')

In [ ]:
!cp /content/vectorizer_ngram.joblib /content/drive/MyDrive/DMA/

In [ ]:
import joblib

# Save the TF-IDF transformed features with ngram_range=(3,3)
joblib.dump(X_train_ngram, 'X_train_ngram.joblib')
joblib.dump(X_test_ngram, 'X_test_ngram.joblib')

In [ ]:
# vectorizer_ngram
joblib.dump(vectorizer_ngram, 'vectorizer_ngram.joblib')

In [ ]:
!cp /content/X_train_ngram.joblib /content/drive/MyDrive/DMA/
!cp /content/X_test_ngram.joblib /content/drive/MyDrive/DMA/

In [ ]:
import joblib

# Load the saved TF-IDF transformed features with TFIDF
X_train_tfidf = joblib.load('X_train_tfidf.joblib')
X_test_tfidf = joblib.load('X_test_tfidf.joblib')

In [ ]:
import joblib

# Load the saved TF-IDF transformed features with ngram_range=(3,3)
X_train_ngram = joblib.load('X_train_ngram.joblib')
X_test_ngram = joblib.load('X_test_ngram.joblib')

In [ ]:
# Train the model using Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

In [ ]:
# Predict and evaluate the model
predictions = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Compute precision
precision = precision_score(y_test, predictions, average='binary')

# Compute recall
recall = recall_score(y_test, predictions, average='binary')

# Compute F1 score
f1 = f1_score(y_test, predictions, average='binary')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
# Train the model using Logistic Regression and ngrams
model_ngrams = LogisticRegression(max_iter=1000)
model_ngrams.fit(X_train_ngram, y_train)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Predict and evaluate the model_ngrams
predictions_ngram = model_ngrams.predict(X_test_ngram)
accuracy = accuracy_score(y_test, predictions_ngram)
precision = precision_score(y_test, predictions_ngram, average='binary')
recall = recall_score(y_test, predictions_ngram, average='binary')
f1 = f1_score(y_test, predictions_ngram, average='binary')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
# save the model
from joblib import dump

dump(model_ngrams, 'logistic_regression_model_ngrams.joblib')

In [ ]:
!cp /content/logistic_regression_model_ngrams.joblib /content/drive/MyDrive/DMA/

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model_nb = MultinomialNB()
model_nb.fit(X_train_tfidf, y_train)

In [ ]:
dump(model_nb, 'naive_bayes_model_tfidf.joblib')

In [ ]:
!cp /content/naive_bayes_model_tfidf.joblib /content/drive/MyDrive/DMA/

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Predict and evaluate the model_nb
predictions_tfidf = model_nb.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, predictions_tfidf)
precision = precision_score(y_test, predictions_tfidf, average='binary')
recall = recall_score(y_test, predictions_tfidf, average='binary')
f1 = f1_score(y_test, predictions_tfidf, average='binary')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:

model_nb_ngrams = MultinomialNB()
model_nb_ngrams.fit(X_train_ngram, y_train)

In [ ]:
dump(model_nb_ngrams, 'naive_bayes_model_ngrams.joblib')

In [ ]:
!cp /content/naive_bayes_model_ngrams.joblib /content/drive/MyDrive/DMA/

In [ ]:
predictions_ngram = model_nb_ngrams.predict(X_test_ngram)
accuracy = accuracy_score(y_test, predictions_ngram)
precision = precision_score(y_test, predictions_ngram, average='binary')
recall = recall_score(y_test, predictions_ngram, average='binary')
f1 = f1_score(y_test, predictions_ngram, average='binary')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model, X_train_tfidf, y_train, cv=5)

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean cross-validation score:", cv_scores.mean())

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Human', 'AI-generated'], yticklabels=['Human', 'AI-generated'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:

# Function to predict new texts
def predict_text(text):
    preprocessed_text = preprocess_text(text)
    vectorized_text = vectorizer.transform([preprocessed_text])
    prediction = model.predict(vectorized_text)
    return "AI-generated" if prediction[0] == 1 else "Human-written"

In [ ]:
test_text = "I wiped perspiration from my face using my arm, craving another beer to quench my thirst. The chainsaw in my hand, my thoughts drifted to WOXO being off the air and the peculiar fogbank's origin – leading to Shaymore (pronounced Shammore locally) and the Arrowhead Project. According to Bill Giosti, the Arrowhead Project was the cause of the so-called Black Spring in Shaymore. In the western part, near the border with Stoneham, a government preserve with security measures existed, but its details remained elusive to me. Although I had never seen it, the Old Shaymore Road ran alongside the government land. The origin of the Arrowhead Project's name and its existence were shrouded in uncertainty. Bill Giosti claimed insider information through his niece at the Continental Phone Company, but details were vague. He asserted the project involved atomic experiments, shooting atoms into the air. Billy challenged this, citing that the air was already full of atoms, to which Bill Giosti insisted they were different. Dick Muehler suggested the project was merely an agricultural station for cultivating larger tomatoes with an extended growing season. Janine Lawless, the postlady, believed it to be a geological survey related to shale oil. Mrs. Carmody, however, aligned more with Bill Giosti's perspective, suggesting a more mysterious involvement with distinct atoms.As I continued cutting a large tree, Billy returned with a fresh beer and a note from Steff. Acknowledging his penchant for delivering messages, I accepted both and cautioned him against getting drunk at such an early hour."
print(len(test_text))
print(predict_text(test_text))

In [ ]:
chatGPT_test_text= "In the quaint town of Willow Creek, nestled amidst rolling hills and whispering pines, there unfolded a love story that spanned decades, weaving its intricate threads through the tapestry of time. It began on a dew-kissed morning in spring when Emily, a spirited artist with a heart as vast as the cerulean skies above, stumbled upon a forgotten bookstore tucked away in a cobblestone alley. There, amidst the musty scent of old books and ink-stained pages, she met Gabriel, a melancholic poet with eyes that held galaxies within. Their initial encounter was brief, a fleeting moment that lingered in the corners of their minds like a whispered promise. Yet, fate had intertwined their destinies, and through a series of serendipitous events, their paths crossed again and again, each meeting deepening the connection between their souls. As seasons danced and years unfurled, Emily and Gabriel journeyed through the highs and lows of life together, their love blossoming like a perennial flower in the garden of their hearts. They shared laughter under the star-studded sky, found solace in each other's embrace during stormy nights, and reveled in the simple joys of everyday moments. But just as in any epic tale, their love was tested by trials and tribulations, challenging their commitment and resilience. Yet, through it all, they remained steadfast, their bond forged in the fires of adversity, emerging stronger and more enduring than ever before. And so, as the sun dipped below the horizon on a golden autumn evening, Emily and Gabriel stood hand in hand, surrounded by loved ones, exchanging vows beneath an ancient oak tree that had witnessed the evolution of their love. For theirs was a love story written in the stars, a timeless saga of passion, perseverance, and the enduring power of love to transcend the boundaries of time and space."

print(len(chatGPT_test_text))
print(predict_text(chatGPT_test_text))

In [ ]:
gemini_test_text = "Eleanor, with her fiery hair and a laugh that could silence a crowded room, first met Ethan on a rainy Tuesday, their hands brushing over a dusty paperback in a cramped bookstore. Years melted into a kaleidoscope of shared dreams - stolen picnics under ancient oaks, whispered secrets under a sky ablaze with stars, the quiet comfort of two souls settling into one. Life wasn't a fairytale - there were arguments that scorched the air and misunderstandings that left a bitter taste, but through it all, their love remained a constant, a flickering candle in the storm. They built a life together, a mismatched quilt woven with threads of laughter, tears, the pitter-patter of tiny feet as their son, Liam, and daughter, Clara, learned to walk, and the bittersweet pang of letting go as Liam, with a mop of his father's dark hair and Eleanor's infectious grin, set off for college, followed a few years later by Clara, whose fiery spirit mirrored her mother's. Silver threads began to paint their hair, their laughter turned gentler, their gazes holding a lifetime of unspoken stories. Yet, even as their bodies aged, their love never faltered. They learned the language of knowing glances, the comfort of a hand held silently, the quiet understanding that bloomed after decades of being one another's confidante, best friend, and lover. They weathered storms, both literal and metaphorical, their love a sturdy oak weathering the harshest winds. There were hospital stays, the fear a cold knot in their chests, but they faced them together, a testament to a love that defied time. On a quiet Monday evening, years after that rainy Tuesday, Eleanor, her hand held tight in Ethan's, closed her eyes for the last time, a faint smile gracing her lips as if she saw something wonderful beyond the veil. Though the world dimmed a little for Ethan, the embers of their love, a love story whispered across decades, would forever glow warm in the chambers of his heart. He carried their story forward, finding solace in memories, in the worn armchair where they'd sat for countless evenings, in the garden brimming with flowers Eleanor had meticulously tended. He knew, with a certainty that transcended words, that their love story, a testament to the enduring power of connection, would echo through eternity. "

print(len(gemini_test_text))
print(predict_text(gemini_test_text))

In [ ]:
gemini_test_text2 = "A chance bookstore touch ignited a love that danced through decades. Laughter, dreams, tears, a son's mop of dark hair, a daughter's fiery spirit - their life a tapestry woven tight. Silver hair, gentle smiles, a love language in shared gazes. They weathered storms, hand in hand, a love stronger than time. One quiet Monday, her eyes closed, a faint smile - a love story whispered across decades, forever warm in his heart."

print(len(gemini_test_text2))
print(predict_text(gemini_test_text2))

In [ ]:
chatGPT_test_text2 = "In the cozy little town of Willow Creek, with its rolling hills and whispering pines, there unfolded a love story that spanned decades, weaving its intricate threads through the fabric of time. It all started one spring morning, when Emily, a vibrant artist with a heart as boundless as the sky above, stumbled upon an old bookstore tucked away in a cobblestone alley. There, amid the scent of aging books and ink-stained pages, she met Gabriel, a brooding poet with eyes that seemed to hold entire universes within them. Their first meeting was fleeting, but it left an indelible mark on their hearts, setting the stage for a series of chance encounters that brought them closer together with each passing moment. Through seasons of joy and sorrow, Emily and Gabriel journeyed side by side, finding solace in each other's arms during the darkest of nights and rejoicing in the simple pleasures of everyday life. Their love endured trials and tribulations, emerging stronger with each obstacle they faced. And so, on a crisp autumn evening, surrounded by family and friends beneath the branches of an ancient oak tree, Emily and Gabriel exchanged vows, sealing their love in a moment that felt as timeless as the stars above. For theirs was a love story written in the very fabric of the universe, a testament to the enduring power of love to conquer all obstacles and stand the test of time."

print(len(chatGPT_test_text2))
print(predict_text(chatGPT_test_text2))

# BERT

In [ ]:
# Preprocessing
X_train_cleaned = np.where(pd.isnull(X_train), '', X_train)
X_test_cleaned = np.where(pd.isnull(X_test), '', X_test)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_cleaned, y_train, test_size=0.1, random_state=42)

In [ ]:
print(len(X_train))
print(len(y_train))

print(len(X_val))
print(len(y_val))


14400
14400
1600
1600


In [ ]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Tokenize the text data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test_cleaned.tolist(), truncation=True, padding=True, max_length=128)

In [ ]:
y_train_int = y_train.astype(int)
y_val_int = y_val.astype(int)


# Convert labels to tensors
y_train_tensor = torch.tensor(y_train_int.tolist())
y_val_tensor = torch.tensor(y_val_int.tolist())

In [ ]:
print(y_train_tensor)

tensor([1, 0, 1,  ..., 1, 1, 0])


In [ ]:
# Convert tokenized inputs to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              y_train_tensor)
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            y_val_tensor)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']))


In [ ]:
# Define the batch size and create data loaders
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model.to(device)

In [ ]:
# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, 1)
            val_preds.extend(predicted_labels.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
    val_accuracy = accuracy_score(val_labels, val_preds)

    print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

In [ ]:
# Testing
model.eval()
test_preds = []
test_texts = []  # Store input text
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, 1)
        test_preds.extend(predicted_labels.cpu().numpy())
        test_texts.extend(tokenizer.batch_decode(input_ids, skip_special_tokens=True))  # Decode input text

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Assuming you have the true labels for your test data stored in `true_labels`
true_labels = torch.tensor(y_test.tolist())

# Convert test_preds to numpy array
test_preds = np.array(test_preds)

# Calculate Accuracy
accuracy = accuracy_score(true_labels, test_preds)

# Calculate F1 Score
f1 = f1_score(true_labels, test_preds, average='weighted')

# Calculate Precision
precision = precision_score(true_labels, test_preds, average='weighted')

# Calculate Recall
recall = recall_score(true_labels, test_preds, average='weighted')

# Print the metrics
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

Accuracy: 0.9655
F1 Score: 0.9654683107484433
Precision: 0.9661740048680276
Recall: 0.9655


In [ ]:
torch.save(model, 'bert_model.pth')

In [ ]:
!cp /content/bert_model.pth /content/drive/MyDrive/DMA/

In [ ]:
"""
model = torch.load('/content/drive/MyDrive/DMA/bert_model.pth')
"""

"\nmodel = torch.load('bert_model.pth')\n"

# Test

In [ ]:
file_path = "human_text.txt"

with open(file_path, "r") as file:
    human_lines = file.readlines()
    human_lines = [preprocess_text(line.strip()) for line in human_lines if line.strip()]

print(len(human_lines))

7


In [ ]:
file_path = "GPT4.txt"

with open(file_path, "r") as file:
    lines = file.readlines()
    lines = [preprocess_text(line.strip()) for line in lines if line.strip()]

print(len(lines))

9


In [ ]:
def average_line_length(lines):
    total_length = sum(len(line) for line in lines)
    num_lines = len(lines)
    average_length = total_length / num_lines if num_lines > 0 else 0
    return average_length

print("GPT4: ",average_line_length(lines))
print("Human: ",average_line_length(human_lines))

GPT4:  1748.5555555555557
Human:  3814.0


In [ ]:
df = pd.DataFrame({'X': human_lines + lines, 'y': [0]*len(human_lines) + [1]*len(lines)})

# Displaying the DataFrame
print(df)

                                                    X  y
0   man found standing alone prairie large strong ...  0
1   dear cassandra letter agreeable surprise day t...  0
2   model general smart grid system considered stu...  0
3   introduction evolutionary ensemble learning em...  0
4   headline appeared opinion page tuesday strikin...  0
5   artificial intelligence ai undoubtedly revolut...  0
6   pay attention enough notice sometimes resist s...  0
7   amidst emerald embrace ancient tree sunlight f...  1
8   climate change critical issue time global phen...  1
9   bermuda triangle region western part north atl...  1
10  quantum entanglement phenomenon intrigued baff...  1
11  intersectional feminism term coined kimberlé c...  1
12  culinary fusion concept revolutionized food in...  1
13  space exploration colonization concept captiva...  1
14  grand scheme thing find middle 20th century be...  1
15  dawn 20th century stand precipice new era worl...  1


## Logistic Regression test

### TFIDF Model

In [ ]:
import joblib

# Load the saved TF-IDF transformed features with TFIDF
X_train_tfidf = joblib.load('/content/drive/MyDrive/DMA/X_train_tfidf.joblib')
X_test_tfidf = joblib.load('/content/drive/MyDrive/DMA/X_test_tfidf.joblib')

In [ ]:
log_model_tfidf = joblib.load('/content/drive/MyDrive/DMA/logistic_regression_model_tfidf.joblib')
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/DMA/tfidf_vectorizer.joblib')

In [ ]:
TFIDF_test = tfidf_vectorizer.transform(df['X'])

In [ ]:
preds = log_model_tfidf.predict(TFIDF_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary', zero_division=1)
    recall = recall_score(y_true, y_pred, average='binary', zero_division=1)
    f1 = f1_score(y_true, y_pred, average='binary')

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

    return accuracy, precision, recall, f1

In [ ]:
print(df['y'])
print(preds)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
Name: y, dtype: int64
[0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
accuracy, precision, recall, f1 = calculate_metrics(df['y'], preds)

Accuracy: 0.75
Precision: 0.6923076923076923
Recall: 1.0
F1 Score: 0.8181818181818181


### NGrams

In [ ]:
import joblib

# Load the saved TF-IDF transformed features with ngrams
X_train_ngrams = joblib.load('/content/drive/MyDrive/DMA/X_train_ngram.joblib')
X_test_ngrams = joblib.load('/content/drive/MyDrive/DMA/X_test_ngram.joblib')

In [ ]:
log_model_ngram = joblib.load('/content/drive/MyDrive/DMA/logistic_regression_model_ngrams.joblib')
ngram_vectorizer = joblib.load('/content/drive/MyDrive/DMA/vectorizer_ngram.joblib')

In [ ]:
ngram_test = ngram_vectorizer.transform(df['X'])
ngram_preds = log_model_ngram.predict(ngram_test)

In [ ]:
print(np.array(df['y']))
print(ngram_preds)

[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
[0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1.]


In [ ]:
accuracy, precision, recall, f1 = calculate_metrics(df['y'], ngram_preds)

Accuracy: 0.625
Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666


## Naive Bayes

### TFIDF

In [ ]:
nb_model_tfidf = joblib.load('/content/drive/MyDrive/DMA/naive_bayes_model_tfidf.joblib')
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/DMA/tfidf_vectorizer.joblib')

In [ ]:
tfidf_test = tfidf_vectorizer.transform(df['X'])
tfidf_preds_nb = nb_model_tfidf.predict(tfidf_test)

In [ ]:
print(np.array(df['y']))
print(tfidf_preds_nb)

[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
[0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
accuracy, precision, recall, f1 = calculate_metrics(df['y'], tfidf_preds_nb)

Accuracy: 0.6875
Precision: 0.6428571428571429
Recall: 1.0
F1 Score: 0.782608695652174


NGrams

In [ ]:
nb_model_ngrams = joblib.load('/content/drive/MyDrive/DMA/naive_bayes_model_ngrams.joblib')

In [ ]:
ngram_test = ngram_vectorizer.transform(df['X'])
nb_ngram_preds = nb_model_ngrams.predict(ngram_test)

In [ ]:
print(np.array(df['y']))
print(nb_ngram_preds)

[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
[0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
accuracy, precision, recall, f1 = calculate_metrics(df['y'], nb_ngram_preds)

Accuracy: 0.8125
Precision: 0.75
Recall: 1.0
F1 Score: 0.8571428571428571


## Bert

In [ ]:
bert_model = torch.load('/content/drive/MyDrive/DMA/bert_model.pth')

In [ ]:
test_encodings = tokenizer(df['X'].tolist(), truncation=True, padding=True, max_length=128)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
# Testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.eval()
test_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, 1)
        test_preds.extend(predicted_labels.cpu().numpy())

In [ ]:
print(np.array(df['y']))
print(test_preds)

[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
[0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1]


In [ ]:
accuracy, precision, recall, f1 = calculate_metrics(df['y'], test_preds)

Accuracy: 0.625
Precision: 0.6363636363636364
Recall: 0.7777777777777778
F1 Score: 0.7000000000000001
